### This very simple notebook will take a blood pressure reading log (the provided example data is produced by an OMRON Smart X reader), allocate the readings to three sections of the day (AM, MID and PM) and draw a boxplot to perform a first rough estimate about the presence of a trend.

With the data provided for instance you can form the hypothesis that the pressure in the afternoon/evening is better that in the morning/central part of the day and for instance you could act shifting the time of your pressure medication from the evening to the morning to better control the daily values.

Import needed libraries

In [ ]:
# One ring to handle numbers
import pandas as pd
import scipy.stats as stats
# One ring to handle time
import datetime as dt
from datetime import date
import time
# One ring to draw reality
import seaborn as sns
import matplotlib.pyplot as plt
# and One ring to rule them all ;)
# from matplotlib.dates import date2num

Set the (arbitrary) limits to divide the day in an AM, MID(day) and PM areas and the name of data file

In [ ]:
AM_LIMIT = 10
PM_LIMIT = 16
# LOGNAME = 'data/bp_log.csv' # data file locally
LOGNAME = '../input/blood-pressure-readings/bp_log.csv' # on Kaggle
# https://www.kaggle.com/rjalexander/blood-pressure-circadian

Define a function to generate a "partofday" categorical label based on the hour of the blood pressure reading and the limits we set above

In [ ]:
def partofday(hour):
    if hour <= AM_LIMIT:
        return 'AM'
    elif hour > AM_LIMIT and hour <= PM_LIMIT:
        return 'MID'
    else:
        return 'PM'

Read and take a peek at the data as read in a dataframe

In [ ]:
df = pd.read_csv(LOGNAME)
df.head()

Add a new "partofday" categorical new column in the dataframe

In [ ]:
df['partofday'] = df.apply(lambda row: partofday(pd.to_datetime(row['Measurement Date']).hour), axis=1)

Now plot the SYS(tolic) and then the DIA(stolic) pressure values divided in the three sections of the day

In [ ]:
sns.set_theme(style="whitegrid")

In [ ]:
ax = sns.boxplot(x=df["partofday"], y=df["SYS"],hue=df["partofday"],data=df, order=["AM","MID","PM"])
ax = sns.swarmplot(x=df["partofday"], y=df["SYS"],hue=df["partofday"],data=df, order=["AM","MID","PM"])
ax.legend_.remove()

In [ ]:
ax = sns.boxplot(x=df["partofday"], y=df["DIA"],hue=df["partofday"],data=df, order=["AM","MID","PM"])
ax = sns.swarmplot(x=df["partofday"], y=df["DIA"],hue=df["partofday"],data=df, order=["AM","MID","PM"])
ax.legend_.remove()

Now let's perform a t-test to see if there's a statistically significant difference in the average BP between AM and PM

In [ ]:
stats.ttest_ind(df['SYS'][df['partofday'] == 'AM'], df['SYS'][df['partofday'] == 'PM'])

In [ ]:
stats.ttest_ind(df['DIA'][df['partofday'] == 'AM'], df['DIA'][df['partofday'] == 'PM'])

Now let's perform a t-test to see if there's a statistically significant difference in the average BP between AM and MIDday

In [ ]:
stats.ttest_ind(df['SYS'][df['partofday'] == 'AM'], df['SYS'][df['partofday'] == 'MID'])

In [ ]:
stats.ttest_ind(df['DIA'][df['partofday'] == 'AM'], df['DIA'][df['partofday'] == 'MID'])

### Conclusion: from this initial assessment of the data you could hypothesize that the pressure is better (lower) in the afternoon/evening and therefore reccomend the patient to take her medication in the morning as opposed to the evening. The difference between AM and MIDday is not significant.

### Let's now plot the BP values and see what's the temporal trend in time

### Add a date_ordinal column converting the datetime value to an epoch (seconds till 01/01/70)


In [ ]:
df['date_ordinal'] = pd.to_datetime(df['Measurement Date']).apply(lambda date: date.timestamp())

Now plot the line graph using "timestamp" datetime as the time-series x value

In [ ]:
plt.figure(figsize=(14,6))
ax1 = df.SYS.plot(x='timestamp', color='red', grid=True, label='SYS')
ax1 = df.DIA.plot(x='timestamp', color='blue', grid=True, label='DIA')
plt.legend()
plt.legend(loc='upper left')
plt.xticks(rotation=25)
ax1.set_xlabel('date (days)')
plt.show()

Now plot a scatterplot with a linear regression fit. To do so as the scattterplot does not accept a datetime timestamp as the x values, I am using the epoch (seconds from 01-jan-1970) contained in the "date_ordinal" column

In [ ]:
plt.figure(figsize=(14,6))
ax = sns.regplot(x="date_ordinal", y="SYS", data=df, label="SYS")
ax = sns.regplot(x="date_ordinal", y="DIA", data=df, label="DIA")
# create some whitespace horizontally around the plot values
ax.set_xlim(df['date_ordinal'].min() - 100000, df['date_ordinal'].max() + 100000)
# now convert the epoch values back to a YYYY-MM-DD string for the x labels
# localtime converts the epoch into a datetime object abd strftime converts it into a string
new_labels = [time.strftime('%Y-%m-%d',time.localtime(item)) for item in ax.get_xticks()]
ax.set_xticklabels(new_labels)
# set some visuals
ax.set_xlabel('date')
plt.xticks(rotation=25)
plt.legend()
plt.show()